In [1]:
import os
from os.path import abspath
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import roc_auc_score

In [2]:
# Set working directory
os.chdir(os.path.dirname(os.getcwd()))
os.getcwd()

'/home/ubuntu/diabetes_detection'

In [3]:
# Import local libraries
from src.logger import set_logger as sl

# Load transformations
from src.transformations.CC_RRC_RVF_RHF_RA import get_transform as gt_train
from src.transformations.CC_R import get_transform as gt_valid
from src.plot_images import plot_images
from src.train_validation import train_validation_phase
from src.test_model import test_phase

In [4]:
# Put data
batch_size    = 64
num_workers   = 8
is_pretrained = False

crop_size     = 1536
is_grayscale  = False

test_pth     = abspath('/mnt/ukbb/old/finetune/')
model_pth    = abspath('/home/ubuntu/sandbox/data/output/models/iv3_ukbb_raw_4da.pth')

# Device setting and CUDA report 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [5]:
dataset = datasets.ImageFolder(test_pth, gt_valid(crop_size, resize=(300,300)))
class_names = dataset.classes
class_names

['female', 'male']

In [6]:
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)
dataloader

In [7]:
# Load model
model = models.inception_v3(pretrained=is_pretrained, init_weights=True, aux_logits=False)

# Get number of features in the model
n_ftrs = model.fc.in_features

# Add linear transformation to the data
model.fc = nn.Linear(n_ftrs, len(class_names))

# Send model to device
model = model.to(device)

In [8]:
results = test_phase(model, model_pth, dataloader, device, logger=None)

In [9]:
results["test_auc"]

0.6821030485785189